In [1]:
import glob
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error , r2_score

car12 = pd.read_excel('2018년 12월 서울시 차량통행속도.xlsx')
print(car12.columns)

Index(['일자', '요일', '도로명', '링크아이디', '시점명', '종점명', '방향', '거리', '차선수', '기능유형구분',
       '도심/외곽구분', '권역구분', '01시', '02시', '03시', '04시', '05시', '06시', '07시',
       '08시', '09시', '10시', '11시', '12시', '13시', '14시', '15시', '16시', '17시',
       '18시', '19시', '20시', '21시', '22시', '23시', '24시'],
      dtype='object')


In [2]:
weather = pd.read_csv('weather_merged.csv')
weather.head(3)

,지점,일시,temperature,wind,rain,일자,district,id,time,snow
0,400,2018-01-01 00:00:00,-0.7,1.1,0.0,2018-01-01,강남구,400,0,0.0
1,400,2018-01-01 01:00:00,-0.9,0.9,0.0,2018-01-01,강남구,400,1,0.0
2,400,2018-01-01 02:00:00,-1.0,1.1,0.0,2018-01-01,강남구,400,2,0.0


In [3]:
weather['date'] = weather['일자'].apply(lambda x: int(x.replace("-","")))
weather['time'] = weather['일시'].apply(lambda x: x[11:13]) + "시"
weather.drop(['지점','일자','일시','id'],axis=1,inplace=True)

In [4]:
weather.head(3)

,temperature,wind,rain,district,time,snow,date
0,-0.7,1.1,0.0,강남구,00시,0.0,20180101
1,-0.9,0.9,0.0,강남구,01시,0.0,20180101
2,-1.0,1.1,0.0,강남구,02시,0.0,20180101


In [5]:
car = pd.melt(car12, id_vars=car12.columns[:12])
car.columns = ['date','days','road_name','link_id','start_point','end_point','direction','road_length','road_num',
                    'road_type','urban_suburb','district','time','value']
car.head(3)

,date,days,road_name,link_id,start_point,end_point,direction,road_length,road_num,road_type,urban_suburb,district,time,value
0,20181201,토,4.19로,1080012200,아카데미하우스,국립4.19묘지,상행,1179,1,보조간선도로,외곽,강북구,01시,NaN
1,20181201,토,4.19로,1080012800,국립4.19묘지,국립4.19묘지입구,상행,352,1,보조간선도로,외곽,강북구,01시,NaN
2,20181201,토,4.19로,1080012700,국립4.19묘지입구,국립4.19묘지,하행,354,2,보조간선도로,외곽,강북구,01시,NaN


In [6]:
car = pd.merge(car,weather,on=['district','time','date'])
car.drop(['date','road_name','start_point','end_point'],axis=1,inplace=True)

In [16]:
car = car[(car['time']!="01시") & (car['time']!="02시") & (car['time']!="03시") & (car['time']!="04시") & (car['time']!="05시")]
car = car.dropna(axis=0)
print(car.shape)
car.head(3)

(2511270, 14)


,days,link_id,direction,road_length,road_num,road_type,urban_suburb,district,time,value,temperature,wind,rain,snow
709179,일,1080012200,상행,1179,1,보조간선도로,외곽,강북구,06시,43.77,2.4,0.7,0.0,0.0
709180,일,1080012800,상행,352,1,보조간선도로,외곽,강북구,06시,21.77,2.4,0.7,0.0,0.0
709181,일,1080012700,하행,354,2,보조간선도로,외곽,강북구,06시,26.80,2.4,0.7,0.0,0.0


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.regplot(x=car["wind"], y=car["value"])
plt.show()

sns.regplot(x=car["rain"], y=car["value"])
plt.show()

In [18]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
car["days"] = lb_make.fit_transform(car["days"])
car["direction"] = lb_make.fit_transform(car["direction"])
car["road_type"] = lb_make.fit_transform(car["road_type"])
car["urban_suburb"] = lb_make.fit_transform(car["urban_suburb"])
car["district"] = lb_make.fit_transform(car["district"])
car["time"] = lb_make.fit_transform(car["time"])

print(car.shape)
car.head()

(2511270, 14)


,days,link_id,direction,road_length,road_num,road_type,urban_suburb,district,time,value,temperature,wind,rain,snow
709179,4,1080012200,0,1179,1,2,1,2,0,43.77,2.4,0.7,0.0,0.0
709180,4,1080012800,0,352,1,2,1,2,0,21.77,2.4,0.7,0.0,0.0
709181,4,1080012700,1,354,2,2,1,2,0,26.80,2.4,0.7,0.0,0.0
709182,4,1080012100,1,1190,2,2,1,2,0,52.60,2.4,0.7,0.0,0.0
709183,4,1080010300,0,868,3,2,1,2,0,24.08,2.4,0.7,0.0,0.0


In [19]:
X_data = car.drop(['link_id','value'],axis=1,inplace=False)
y_target = car['value']
X_train , X_test , y_train , y_test = train_test_split(X_data , y_target ,test_size=0.3, random_state=156)

In [20]:
def get_model_predict(model, X_train, X_test, y_train, y_test, is_expm1=False):
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    if is_expm1 :
        y_test = np.expm1(y_test)
        pred = np.expm1(pred)
    mse = mean_squared_error(y_test, pred)    
    r2 = r2_score(y_test, pred)   
    
    print('###',model.__class__.__name__,'###')
    print('MSE: {0:.3f}, R2: {1:.3F}'.format(mse, r2*100))

In [21]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression(n_jobs=-1)
get_model_predict(lr,X_train, X_test, y_train, y_test)

### LinearRegression ###
MSE: 132.951, R2: 17.872


In [22]:
y_target_log = np.log1p(y_target)
X_train , X_test , y_train , y_test = train_test_split(X_data , y_target_log ,test_size=0.3, random_state=156)

lr = LinearRegression(n_jobs=-1)
get_model_predict(lr,X_train, X_test, y_train, y_test,is_expm1=True)

### LinearRegression ###
MSE: 137.558, R2: 15.026


In [27]:
X_data_ohe = pd.get_dummies(X_data, columns=['days','time','urban_suburb','road_type','direction'])

X_train , X_test , y_train , y_test = train_test_split(X_data_ohe , y_target_log ,test_size=0.3, random_state=156)

lr = LinearRegression(n_jobs=-1)
get_model_predict(lr,X_train, X_test, y_train, y_test,is_expm1=True)

### LinearRegression ###
MSE: 83.635, R2: 48.336


In [28]:
X_train , X_test , y_train , y_test = train_test_split(X_data_ohe , y_target,test_size=0.3, random_state=156)

lr = LinearRegression(n_jobs=-1)
get_model_predict(lr,X_train, X_test, y_train, y_test,is_expm1=False)

### LinearRegression ###
MSE: 82.157, R2: 49.250


In [29]:
X_train.shape

(1757889, 39)